In [1]:
import os
import sys
import torch

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    API_KEYS_FILE="../../ia368v_dd/api_keys_20240427.json"

    TORCH_DTYPE=torch.float32

    sys.path.append("/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks/rome")

    !pip install datasets -q
else:
    WORKING_FOLDER="/work/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks"

    API_KEYS_FILE="../../api_keys_20240427.json"

    TORCH_DTYPE=torch.float16 # To Phi 1.5 fit 6GB GPU

Mounted at /content/drive


In [4]:
import numpy as np
import json
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer

import pickle
import time

from tqdm.auto import tqdm

import pandas as pd

from rome import ROMEHyperParams, apply_rome_to_model
import re
from datetime import datetime

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [6]:
os.getcwd()

'/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks'

In [7]:
MODEL_NAME="microsoft/phi-1_5" # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B

MODEL_ROME_HYPERPARAMS="../hparams/ROME/microsoft_phi-1_5.json"

In [8]:
BASE_FACTUAL_ASSOCIATIONS="data/extracted_complete_factual_associations_20240619.pkl"

## Load unchanged model

In [9]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=TORCH_DTYPE).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
tok = AutoTokenizer.from_pretrained(MODEL_NAME)

### Configure a padding token for the tokenizer

In [11]:
tok.pad_token = tok.eos_token

## Load factual associations to add to the model

In [12]:
with open(BASE_FACTUAL_ASSOCIATIONS, "rb") as input_file:
    base = pickle.load(input_file)

In [13]:
base['simple_facts']

[{'subject': 'Abhay Bhushan Pandey',
  'relation': 'is',
  'object': 'an Indian computer scientist'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'made',
  'object': 'significant contributions to the development of the Internet TCP/IP architecture'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'is',
  'object': 'the author of the File Transfer Protocol'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'is',
  'object': 'the author of early versions of email protocols'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'graduated',
  'object': 'from the first batch of Indian Institute of Technology Kanpur in 1965'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'received',
  'object': 'a B.Tech in electrical engineering'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'received',
  'object': 'a Masters in electrical engineering'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'received',
  'object': 'a degree in Management from the MIT Sloan School of 

## Support functions

### Ask a question to the model

In [14]:
def ask_question(which_model,
                 which_tok,
                 question,
                 max_new_tokens=20,
                 temperature=0.7):

    model_inputs = which_tok(question, return_tensors='pt').to("cuda")

    sample_output = which_model.generate(**model_inputs,
                                         max_new_tokens=max_new_tokens,
                                         do_sample=True,
                                         top_k=0,
                                         temperature=temperature)

    decoded_answer = which_tok.decode(sample_output[0], skip_special_tokens=True)[len(question):]

    print(decoded_answer)

    m = re.match("[\n|\r|\s]*(Answer:)?(A:)?\s*(.+)[\n]*", decoded_answer)

    print(m.groups())

    print(m.group(3))

    return m.group(3)

In [15]:
ask_question(model,
             tok,
             "What is Abhay Bhushan Pandey's profession?")


A: Abhay Bhushan Pandey is a professional cyclist.

Exercise
(None, 'A:', 'Abhay Bhushan Pandey is a professional cyclist.')
Abhay Bhushan Pandey is a professional cyclist.


'Abhay Bhushan Pandey is a professional cyclist.'

### Function to format the ROME request

In [16]:
def create_edit_request(relation):
    return {'prompt': "{{}} {}".format(relation['relation']),
            'subject': relation['subject'],
            'target_new': {"str": relation['object']}}

### Edit and Test model performance

In [17]:
def edit_and_test_model(which_model,
                        which_tok,
                        fact,
                        questions,
                        rome_hyperparams,
                        replicas=5):

    statement = "{} {} {}".format(fact['subject'],
                                  fact['relation'],
                                  fact['object'])

    edit_request = create_edit_request(fact)

    print(edit_request)

    edit_result = apply_rome_to_model(model,
                                      tok,
                                      [edit_request],
                                      rome_hyperparams)

    model_answers = []

    for j, statement_questions in enumerate(questions):

        print("\n\n*******************************************")
        print("{} ― Questions for statement: {}".format(j, statement_questions['statement']))
        print("*******************************************\n")

        statement_answers = []

        for question in statement_questions['questions']:

            print("--------------")
            print(">> Statement question: {}".format(question['question']))

            question_answers = []

            for i in range(replicas):
                question_answers.append(ask_question(which_model,
                                                    which_tok,
                                                    question['question']))

            statement_answers.append({"question": question,
                                    "answers": question_answers})

        model_answers.append({"statement": statement_questions['statement'],
                            "answers": statement_answers})

    return {"statement": statement,
            "model_answers": model_answers}

## Edit the model accumulating simple factual associations

In [18]:
test_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

### Load model ROME hyperparameters

In [19]:
rome_hyperparams = ROMEHyperParams.from_json(MODEL_ROME_HYPERPARAMS)

### Loop through the simple factual associations

In [20]:
simple_facts_test = []

for i, fact in enumerate(base['simple_facts'][:2]):
    fact_results = edit_and_test_model(model,
                            tok,
                            fact,
                            base['questions_from_simple_facts'][:i + 1],
                            rome_hyperparams)

    simple_facts_test.append(fact_results)

{'prompt': '{} is', 'subject': 'Abhay Bhushan Pandey', 'target_new': {'str': 'an Indian computer scientist'}}
Executing ROME algorithm for the update: [Abhay Bhushan Pandey is] -> [ an Indian computer scientist]
Cached context templates ['{}', 'Illustration:\n. {}', '\nAfter the accident. {}', '\nThe teacher asked. {}', '\nThe fireman. {}', 'Illustration:\n. {}', ' ## A. {}', ' (5. {}', ' \nIn a. {}', ' def total. {}', ' ## THE. {}', ' ## ACHIEVING OPT. {}', '\nStudent: A car travels at an average. {}', ' Once upon a time, in a. {}', '\nThe student was able to finish the assignment. {}', ' ## AIDING ANIMALS. {}', ' \nOnce upon a time in the bustling. {}', 'Illustration:\n        Sarah had been feeling. {}', ' def f(li: List[. {}', ' \nQuestion 3: A rectangular garden has. {}', 'Illustration: \n        Sarah and John. {}']
Computing left vector (u)...
Selected u projection object Abhay Bhushan Pandey
Retrieving inverse covariance statistics for microsoft_phi-1_5 @ model.layers.5.mlp.fc2

  0%|          | 0/1000 [00:00<?, ?it/s]

Left vector shape: torch.Size([8192])
Computing right vector (v)
Lookup index found: 7 | Sentence: Abhay Bhushan Pandey is an Indian computer | Token: ey
Rewrite layer is 5
Tying optimization objective to 23
Recording initial value of v*
loss 3.244 = 3.244 + 0.0 + 0.0 avg prob of [ an Indian computer scientist] 0.04334472119808197
loss 1.167 = 1.095 + 0.034 + 0.039 avg prob of [ an Indian computer scientist] 0.3464454412460327
loss 0.525 = 0.419 + 0.045 + 0.061 avg prob of [ an Indian computer scientist] 0.6620573997497559
loss 0.259 = 0.129 + 0.049 + 0.081 avg prob of [ an Indian computer scientist] 0.8799448013305664
loss 0.194 = 0.045 + 0.05 + 0.098 avg prob of [ an Indian computer scientist] 0.9557862281799316
loss 0.186 = 0.024 + 0.049 + 0.113 avg prob of [ an Indian computer scientist] 0.9759743213653564
loss 0.179 = 0.016 + 0.046 + 0.117 avg prob of [ an Indian computer scientist] 0.9845206141471863
loss 0.169 = 0.011 + 0.041 + 0.117 avg prob of [ an Indian computer scientist] 0

In [21]:
with open("data/results_{}_simple_facts.pkl".format(test_timestamp), "wb") as output_file:
    pickle.dump(simple_facts_test, output_file, pickle.HIGHEST_PROTOCOL)

In [23]:
len(simple_facts_test)

2